In [4]:
import pandas as pd
import numpy as np
import sys 
import datetime
import os
import matplotlib as plt
import geopandas as gp

%matplotlib inline

In [5]:
agg = {'Fare':'mean','Trip Total':'mean','Trip Miles':'sum','Trips Pooled':'sum','PRIVATE_TRIPS':'sum','SHARED_TRIPS':'sum','TRIPS':'sum'}

In [6]:
def clean_float_cols(x):
    x = x.replace(',','')
    x = float(x)
    
    return x

In [7]:
def processRawData(infile, outfile):
        """
        Reads data, cleans it, processes it, and writes it to an HDF5 file.
        
        infile  - infile name, raw CSV format
        outfile - output file name, h5 format
        outkey = name of table in output h5 file
        """
        
        print(datetime.datetime.now().ctime(), 'Converting raw data in file: ', infile)
        
        # set up the reader
        reader = pd.read_csv(infile,  
                         iterator = True, 
                         sep = ',', 
        parse_dates = ['Trip Start Timestamp','Trip End Timestamp'], infer_datetime_format = True, chunksize= 25000)

        # establish the writer and clear any table with that file name
        store = pd.HDFStore(outfile)

        # iterate through chunk by chunk so the computer doesn't run out of memory
        rowsRead    = 0
        rowsWritten_weekday = 0
        rowsWritten_weekend = 0
        
        for chunk in reader:   

            rowsRead += len(chunk)
            
            #convert columns from a string because it caused problems
            convert_cols = ['Trip Seconds','Trip Miles', 'Pickup Census Tract', 'Dropoff Census Tract','Fare', 
                            'Tip','Additional Charges', 'Trip Total', 'Trips Pooled']
        
            for column in convert_cols:
                chunk[column] = chunk[column].astype(str).apply(lambda x: x.replace(',',''))
                chunk[column] = chunk[column].astype(float)
           
            chunk['YEAR'], chunk['MONTH'], chunk['DOW'], chunk['HOUR'] = chunk['Trip Start Timestamp'].dt.year, chunk['Trip Start Timestamp'].dt.month, chunk['Trip Start Timestamp'].dt.weekday, chunk['Trip Start Timestamp'].dt.hour
            
            chunk_weekday = chunk[chunk['DOW'].isin([0,1,2,3,4])]
            chunk_weekend = chunk[chunk['DOW'].isin([5,6])]

            chunk_weekday_1 = chunk_weekday[chunk_weekday['HOUR'].isin([22,23,24,1,2,3,4,5])]
            chunk_weekday_2 = chunk_weekday[chunk_weekday['HOUR'].isin([6,7,8])]
            chunk_weekday_3 = chunk_weekday[chunk_weekday['HOUR'].isin([9,10,11,12,13,14,15])]
            chunk_weekday_4 = chunk_weekday[chunk_weekday['HOUR'].isin([16,17,18])]
            chunk_weekday_5 = chunk_weekday[chunk_weekday['HOUR'].isin([19,20,21])]

            chunk_weekend_1 = chunk_weekend[chunk_weekend['HOUR'].isin([22,23,24,1,2,3,4,5])]
            chunk_weekend_2 = chunk_weekend[chunk_weekend['HOUR'].isin([6,7,8])]
            chunk_weekend_3 = chunk_weekend[chunk_weekend['HOUR'].isin([9,10,11,12,13,14,15])]
            chunk_weekend_4 = chunk_weekend[chunk_weekend['HOUR'].isin([16,17,18])]
            chunk_weekend_5 = chunk_weekend[chunk_weekend['HOUR'].isin([19,20,21])]

            
            
            # write the data
            store.append('Weekday_1', chunk_weekday_1, data_columns = True)
            store.append('Weekday_2', chunk_weekday_2, data_columns = True)
            store.append('Weekday_3', chunk_weekday_3, data_columns = True)
            store.append('Weekday_4', chunk_weekday_4, data_columns = True)
            store.append('Weekday_5', chunk_weekday_5, data_columns = True)


            store.append('Weekend_1', chunk_weekend_1, data_columns = True)
            store.append('Weekend_2', chunk_weekend_2, data_columns = True)
            store.append('Weekend_3', chunk_weekend_3, data_columns = True)
            store.append('Weekend_4', chunk_weekend_4, data_columns = True)
            store.append('Weekend_5', chunk_weekend_5, data_columns = True)


            rowsWritten_weekday += len(chunk_weekday)
            rowsWritten_weekend += len(chunk_weekend)

            print ('Read %i rows and kept %i rows in weekday TNC table' % (rowsRead, rowsWritten_weekday))
            print ('kept ' + str(rowsWritten_weekend) + ' rows in weekend TNC table')

        store.close()
        print('Complete!!!')

In [13]:
#create the shared TNC trip h5 table
processRawData('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Transportation_Network_Providers_-_Trips.csv', 'C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Chicago_TNC_Trips_20.H5')

Mon Nov 30 17:07:48 2020 Converting raw data in file:  C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Transportation_Network_Providers_-_Trips.csv
Read 25000 rows and kept 16947 rows in weekday TNC table
kept 8053 rows in weekend TNC table
Read 50000 rows and kept 33879 rows in weekday TNC table
kept 16121 rows in weekend TNC table
Read 75000 rows and kept 50772 rows in weekday TNC table
kept 24228 rows in weekend TNC table
Read 100000 rows and kept 67699 rows in weekday TNC table
kept 32301 rows in weekend TNC table
Read 125000 rows and kept 84660 rows in weekday TNC table
kept 40340 rows in weekend TNC table
Read 150000 rows and kept 101566 rows in weekday TNC table
kept 48434 rows in weekend TNC table
Read 175000 rows and kept 118540 rows in weekday TNC table
kept 56460 rows in weekend TNC table
Read 200000 rows and kept 135567 rows in weekday TNC table
kept 64433 rows in weekend TNC table
Read 225000 rows and kept 152576 rows in weekday TNC table
kept 72424 rows in weeke

Read 2100000 rows and kept 1415800 rows in weekday TNC table
kept 684200 rows in weekend TNC table
Read 2125000 rows and kept 1432335 rows in weekday TNC table
kept 692665 rows in weekend TNC table
Read 2150000 rows and kept 1449063 rows in weekday TNC table
kept 700937 rows in weekend TNC table
Read 2175000 rows and kept 1465715 rows in weekday TNC table
kept 709285 rows in weekend TNC table
Read 2200000 rows and kept 1482288 rows in weekday TNC table
kept 717712 rows in weekend TNC table
Read 2225000 rows and kept 1498979 rows in weekday TNC table
kept 726021 rows in weekend TNC table
Read 2250000 rows and kept 1515523 rows in weekday TNC table
kept 734477 rows in weekend TNC table
Read 2275000 rows and kept 1532282 rows in weekday TNC table
kept 742718 rows in weekend TNC table
Read 2300000 rows and kept 1548990 rows in weekday TNC table
kept 751010 rows in weekend TNC table
Read 2325000 rows and kept 1565521 rows in weekday TNC table
kept 759479 rows in weekend TNC table
Read 23500

Read 4175000 rows and kept 2795173 rows in weekday TNC table
kept 1379827 rows in weekend TNC table
Read 4200000 rows and kept 2811693 rows in weekday TNC table
kept 1388307 rows in weekend TNC table
Read 4225000 rows and kept 2828316 rows in weekday TNC table
kept 1396684 rows in weekend TNC table
Read 4250000 rows and kept 2845026 rows in weekday TNC table
kept 1404974 rows in weekend TNC table
Read 4275000 rows and kept 2861770 rows in weekday TNC table
kept 1413230 rows in weekend TNC table
Read 4300000 rows and kept 2878419 rows in weekday TNC table
kept 1421581 rows in weekend TNC table
Read 4325000 rows and kept 2894799 rows in weekday TNC table
kept 1430201 rows in weekend TNC table
Read 4350000 rows and kept 2911335 rows in weekday TNC table
kept 1438665 rows in weekend TNC table
Read 4375000 rows and kept 2927895 rows in weekday TNC table
kept 1447105 rows in weekend TNC table
Read 4400000 rows and kept 2944494 rows in weekday TNC table
kept 1455506 rows in weekend TNC table


Read 6225000 rows and kept 4158854 rows in weekday TNC table
kept 2066146 rows in weekend TNC table
Read 6250000 rows and kept 4175436 rows in weekday TNC table
kept 2074564 rows in weekend TNC table
Read 6275000 rows and kept 4191931 rows in weekday TNC table
kept 2083069 rows in weekend TNC table
Read 6300000 rows and kept 4208633 rows in weekday TNC table
kept 2091367 rows in weekend TNC table
Read 6325000 rows and kept 4225313 rows in weekday TNC table
kept 2099687 rows in weekend TNC table
Read 6350000 rows and kept 4241888 rows in weekday TNC table
kept 2108112 rows in weekend TNC table
Read 6375000 rows and kept 4258490 rows in weekday TNC table
kept 2116510 rows in weekend TNC table
Read 6400000 rows and kept 4275175 rows in weekday TNC table
kept 2124825 rows in weekend TNC table
Read 6425000 rows and kept 4291843 rows in weekday TNC table
kept 2133157 rows in weekend TNC table
Read 6450000 rows and kept 4308559 rows in weekday TNC table
kept 2141441 rows in weekend TNC table


Read 8275000 rows and kept 5541373 rows in weekday TNC table
kept 2733627 rows in weekend TNC table
Read 8300000 rows and kept 5557857 rows in weekday TNC table
kept 2742143 rows in weekend TNC table
Read 8325000 rows and kept 5574338 rows in weekday TNC table
kept 2750662 rows in weekend TNC table
Read 8350000 rows and kept 5590838 rows in weekday TNC table
kept 2759162 rows in weekend TNC table
Read 8375000 rows and kept 5607413 rows in weekday TNC table
kept 2767587 rows in weekend TNC table
Read 8400000 rows and kept 5623943 rows in weekday TNC table
kept 2776057 rows in weekend TNC table
Read 8425000 rows and kept 5640350 rows in weekday TNC table
kept 2784650 rows in weekend TNC table
Read 8450000 rows and kept 5656814 rows in weekday TNC table
kept 2793186 rows in weekend TNC table
Read 8475000 rows and kept 5673286 rows in weekday TNC table
kept 2801714 rows in weekend TNC table
Read 8500000 rows and kept 5689619 rows in weekday TNC table
kept 2810381 rows in weekend TNC table


...

Read 144100000 rows and kept 98266866 rows in weekday TNC table
kept 45833134 rows in weekend TNC table
Read 144125000 rows and kept 98283866 rows in weekday TNC table
kept 45841134 rows in weekend TNC table
Read 144150000 rows and kept 98300861 rows in weekday TNC table
kept 45849139 rows in weekend TNC table
Read 144175000 rows and kept 98317868 rows in weekday TNC table
kept 45857132 rows in weekend TNC table
Read 144200000 rows and kept 98334808 rows in weekday TNC table
kept 45865192 rows in weekend TNC table
Read 144225000 rows and kept 98351896 rows in weekday TNC table
kept 45873104 rows in weekend TNC table
Read 144250000 rows and kept 98368864 rows in weekday TNC table
kept 45881136 rows in weekend TNC table
Read 144275000 rows and kept 98385861 rows in weekday TNC table
kept 45889139 rows in weekend TNC table
Read 144300000 rows and kept 98402876 rows in weekday TNC table
kept 45897124 rows in weekend TNC table
Read 144325000 rows and kept 98419793 rows in weekday TNC table


Read 146075000 rows and kept 99606925 rows in weekday TNC table
kept 46468075 rows in weekend TNC table
Read 146100000 rows and kept 99623794 rows in weekday TNC table
kept 46476206 rows in weekend TNC table
Read 146125000 rows and kept 99640744 rows in weekday TNC table
kept 46484256 rows in weekend TNC table
Read 146150000 rows and kept 99657696 rows in weekday TNC table
kept 46492304 rows in weekend TNC table
Read 146175000 rows and kept 99674566 rows in weekday TNC table
kept 46500434 rows in weekend TNC table
Read 146200000 rows and kept 99691484 rows in weekday TNC table
kept 46508516 rows in weekend TNC table
Read 146225000 rows and kept 99708432 rows in weekday TNC table
kept 46516568 rows in weekend TNC table
Read 146250000 rows and kept 99725479 rows in weekday TNC table
kept 46524521 rows in weekend TNC table
Read 146275000 rows and kept 99742340 rows in weekday TNC table
kept 46532660 rows in weekend TNC table
Read 146300000 rows and kept 99759178 rows in weekday TNC table


Read 148050000 rows and kept 100945154 rows in weekday TNC table
kept 47104846 rows in weekend TNC table
Read 148075000 rows and kept 100962310 rows in weekday TNC table
kept 47112690 rows in weekend TNC table
Read 148100000 rows and kept 100979215 rows in weekday TNC table
kept 47120785 rows in weekend TNC table
Read 148125000 rows and kept 100996172 rows in weekday TNC table
kept 47128828 rows in weekend TNC table
Read 148150000 rows and kept 101013085 rows in weekday TNC table
kept 47136915 rows in weekend TNC table
Read 148175000 rows and kept 101030017 rows in weekday TNC table
kept 47144983 rows in weekend TNC table
Read 148200000 rows and kept 101046971 rows in weekday TNC table
kept 47153029 rows in weekend TNC table
Read 148225000 rows and kept 101063846 rows in weekday TNC table
kept 47161154 rows in weekend TNC table
Read 148250000 rows and kept 101080925 rows in weekday TNC table
kept 47169075 rows in weekend TNC table
Read 148275000 rows and kept 101097926 rows in weekday 

Read 150025000 rows and kept 102284447 rows in weekday TNC table
kept 47740553 rows in weekend TNC table
Read 150050000 rows and kept 102301358 rows in weekday TNC table
kept 47748642 rows in weekend TNC table
Read 150075000 rows and kept 102318245 rows in weekday TNC table
kept 47756755 rows in weekend TNC table
Read 150100000 rows and kept 102335159 rows in weekday TNC table
kept 47764841 rows in weekend TNC table
Read 150125000 rows and kept 102352063 rows in weekday TNC table
kept 47772937 rows in weekend TNC table
Read 150150000 rows and kept 102368998 rows in weekday TNC table
kept 47781002 rows in weekend TNC table
Read 150175000 rows and kept 102385887 rows in weekday TNC table
kept 47789113 rows in weekend TNC table
Read 150200000 rows and kept 102402765 rows in weekday TNC table
kept 47797235 rows in weekend TNC table
Read 150225000 rows and kept 102419642 rows in weekday TNC table
kept 47805358 rows in weekend TNC table
Read 150250000 rows and kept 102436683 rows in weekday 

Read 152000000 rows and kept 103623113 rows in weekday TNC table
kept 48376887 rows in weekend TNC table
Read 152025000 rows and kept 103640162 rows in weekday TNC table
kept 48384838 rows in weekend TNC table
Read 152050000 rows and kept 103656954 rows in weekday TNC table
kept 48393046 rows in weekend TNC table
Read 152075000 rows and kept 103673819 rows in weekday TNC table
kept 48401181 rows in weekend TNC table
Read 152100000 rows and kept 103690820 rows in weekday TNC table
kept 48409180 rows in weekend TNC table
Read 152125000 rows and kept 103707578 rows in weekday TNC table
kept 48417422 rows in weekend TNC table
Read 152150000 rows and kept 103724573 rows in weekday TNC table
kept 48425427 rows in weekend TNC table
Read 152175000 rows and kept 103741457 rows in weekday TNC table
kept 48433543 rows in weekend TNC table
Read 152200000 rows and kept 103758420 rows in weekday TNC table
kept 48441580 rows in weekend TNC table
Read 152225000 rows and kept 103775340 rows in weekday 